# Assignment: A Smarter Waiting Room

## Learning goals

By the end, you will be able to:

- Model a real-world queue with a 2-D circular buffer.
- Maintain **front/back** indices and **invariants** for a queue without physical shifting.
- Design a focused unit-test suite (wrap-around, full/empty, iteration order).

---

## Scenario (motivation)

A clinic uses an $n	\times n$ waiting room laid out in rows. New arrivals sit in the **first available seat** scanning **front→back, left→right** (row-major order). The **next person called** is always at the **front-left** seat.

**Naïve policy (wasteful):**

- When the front-left person leaves, **everyone shifts one seat left**; gaps at the right edge are back-filled by people from the next row; the last occupied seat becomes empty.
- This causes approximately $n^2$ seat moves in the worst case scenario (in computer science, we use the notation $\mathcal O(n^2)$ to indicate the approximate duration of a worst case scenario).

**Smart receptionist:**
A smart receptionist at the clinic realises that there is no need to have people move to another seat everytime the next person is called. Instead, it is sufficient to track only **who arrived last** and **who is next**. Conceptually, we use the $n\times n$ grid as a seating area where a **front of queue** and **back of queue** pointer advance in row-major order (wrapping around).

You are given a starter implementation that embodies this idea. below.


In [3]:
class TwoDimensionalQ:
    def __init__(self, n: int = 4):
        self._underlying: list[list[str]] = [[None for _ in range(n)] for _ in range(n)]
        self._n: int = n
        self._capacity: int = n * n
        self._usage: int = 0
        self._front_row: int = 0
        self._front_col: int = 0
        self._back_row: int = 0
        self._back_col: int = 0

## What you must do

### Read & explain the data model (short write-up)

In 100-200 words, explain how **row-major order** (front→back, left→right) is implemented using:

- a 2-D array `self._underlying`,
- a pair of indices `self._front`, `self._back`,
- modular arithmetic to **advance and wrap**.

### Implement the queue

Implement the queue so that it is **correct, side-effect free**, and fast. Fast here means that theres should be no shifting of elements closer to the front of the queue every time the next-in-line element is removed.

#### Required behaviors & constraints

- `enqueue(value: str) -> bool`:
  - Return `False` if full; otherwise place at `back`, advance `back` in row-major order with wrap, increment usage.
  - **Do not** mutate other seats; **no prints**.
- `dequeue() -> str | None`:
  - Return `None` if empty; otherwise remove at `front`, set that cell to `None`, advance `front` with wrap, decrement usage.
  - **No prints**; exactly one return.
- `peek() -> str | None`: constant-time look at front.
- `list_queue() -> list[str]`: return logical order of occupants starting at `front` over `usage` items. Must handle wrap correctly and **not** expose `None` elements.
- ensure type hints are consistent.
- implement special methods (dunders) for `__repr__` and `__bool__`.
- implement getters for `usage` and `capacity`.

#### Implementation notes

- You _may_ add private helper methos as you see fit,
- You may not use the `import` statement.
- Methods that return values should have one and only one `return` statement. Methods that do not return values should have no `return` statements.

#### Invariants

- $0 \le 	\texttt{usage} \le 	\texttt{capacity}$.
- For all occupied positions along the logical queue, cells are non-`None`; all other cells are `None`.
- If `usage == 0` then `front == back`.  
  If `usage == capacity` then `front == back` as well (full cycle).

## Deliverables

**Code**: `two_dimensional_q.py` with your finished class and some basic testing showing it works as expected.

---

## Rubric (20 points)

- **Correct queue ops & invariants** (8): all ops done in fixed time with correct wrap, no side effects, invariants hold.
- **Tests** (5): coverage of edge cases & wrap-around; readable assertions.
- **Analysis** (5): clarity of explanations; sound complexity & FIFO argument; simulation results.
- **Style** (2): type hints, docstrings, naming, no duplicate dunders.


---
# SOLUTION & TECHNICAL NOTES
---

The starter code had a magic value (`n=4`) in the `__init__` arguments. This magic value was delegated to constant `_DEFAULT_SIZE` in the solution below.

The core lesson in this assignment is to recognize that a queue is defined by its first and last elements. As long as we know where they are, there is no need for shifting elements around every time the first element is removed from the queue. Tracking first and last element is accomplished by variables `_front` and `_back`. These are pointers to the corresponding position in an $n\times n$ list.

Every time the _first_ item of the queue is removed, the pointer `_front` moves to the next position with a `_front += 1` operation. This, of course, may lead to an out-of-bounds index for the underlying array, so we use modulo (aka modular) arithmetic to ensure a proper wrap up and advance to the next row. As long as we allow additions to the queue while there is room (`_capacity`), this advancing and wrapping the `_front` pointer is guaranteed to work.

The solution below uses tuples to store the front and back indices, mainly to illustrate how this data structure can be used in context. Your own solutions likely use two separate `int` variables for the front (row and column) and two for the back, which is perfectly fine—and arguably clearer. In fact, using individual integer variables often makes the code easier to read and reason about than the more compact, "Pythonic" tuple-based approach.


In [8]:
# 1234567890123456789012345678901234567890123456789012345678901234567890123456789

class TwoDimensionalQ:

    _DEFAULT_SIZE = 4  # ELIMINATE MAGIC VALUE 4 IN __INIT__

    def __init__(self, n: int = _DEFAULT_SIZE):
        """A circular queue implements on an n x n grid modeling the seating
        in a waiting room.
        """
        self._underlying: list[list[str]] = [[None for _ in range(n)] 
                                             for _ in range(n)]
        self._n: int = n
        self._capacity: int = n * n
        self._usage: int = 0
        # (row, column) grid reference for front and back of queue
        self._front: tuple[int, int] = (0, 0)
        self._back: tuple[int, int] = (0, 0)

    # --- Accessors ----

    def get_usage(self) -> int:
        return self._usage

    def get_capacity(self) -> int:
        return self._capacity

    def is_full(self) -> bool:
        return self._usage == self._capacity

    def is_empty(self) -> bool:
        return self._usage == 0

    def peek(self) -> str:
        front_row, front_col = self._front
        return (self._underlying[front_row][front_col] 
                if self._usage > 0 else None)

    # --- Overloading some special methods ---

    def __len__(self) -> int:
        return self.get_usage()

    def __bool__(self) -> bool:
        return not self.is_empty()

    def __repr__(self) -> str:
        return (f"Queue size {self._n}x{self._n}; capacity: "
                f"{self._capacity}; usage:{self._usage}; front is at "
                f"{self._front}; back is at {self._back}")

    def __str__(self) -> str:
        """It's not always a good idea to have the same __repr__ and 
        __str__ as they serve different roles. __str__ is for readability, 
        __repr__ is for debugging. In this case and for simplicity, 
        it's fine to keep them the same."""
        return self.__repr__()

    def __iter__(self):
        """Yield occupants from front to back in FIFO order 
        (handles wrap-around)."""
        count = 0
        row, col = self._front
        while count < self._usage:
            value = self._underlying[row][col]
            # Occupied cells are non-None by invariant; guard defensively anyway.
            if value is not None:
                yield value
            col = (col + 1) % self._n
            row = (row + 1) % self._n if col == 0 else row
            count += 1

    # --- Core functionality ---

    def enqueue(self, value: str) -> bool:
        """Add value to the back of the queue. Return True if successful, 
        False if not."""
        success: bool = self._usage < self._capacity
        if success:
            # There is room to add the value, obtain the back position.
            back_row, back_col = self._back
            # Place the value at the back position and increment usage.
            self._underlying[back_row][back_col] = value
            self._usage += 1
            # Update the back position to the next available slot.
            back_col = (back_col + 1) % self._n
            back_row = (back_row + 1) % self._n if back_col == 0 else back_row
            self._back = (back_row, back_col)
        return success

    def dequeue(self) -> str:
        """Remove and return the value at the front of the queue. If the queue 
        is empty, return None."""
        result = None
        if self._usage > 0:
            # There is something to remove, obtain the front position.
            front_row, front_col = self._front
            result = self._underlying[front_row][front_col]
            # Remove the value at the front position and decrement usage.
            self._underlying[front_row][front_col] = None
            self._usage -= 1
            # Update the front position to the next occupied slot.
            front_col = (front_col + 1) % self._n
            front_row = front_row + 1 if front_col == 0 else front_row
            self._front = (front_row, front_col)
        return result

    def list_queue(self) -> list[str]:
        """Return a list of the items in the queue from front to back."""
        queue: list[str] = list()
        # Traverse the queue from front to back, wrapping as necessary.
        front_row, front_col = self._front
        for i in range(self._usage):
            # Append the current front item to the result list.
            queue.append(self._underlying[front_row][front_col])
            # Move to the next item in the queue.
            front_col = (front_col + 1) % self._n
            front_row = (front_row + 1) % self._n if front_col == 0 else front_row
        return queue


if __name__ == "__main__":
    test = TwoDimensionalQ()
    items_to_add = ["Alice", "Bob", "Cathy", "Derek", "Eva"]
    for item in items_to_add:
        print(test.enqueue(item))

    print(test.list_queue())
    print(test.dequeue())
    print(f"Peeking: {test.peek()}")
    print(test.list_queue())

True
True
True
True
True
['Alice', 'Bob', 'Cathy', 'Derek', 'Eva']
Alice
Peeking: Bob
['Bob', 'Cathy', 'Derek', 'Eva']


In the solution above you can see that the "advance and wrap" operations appear in multiple locations. I left them in place as a discussion point: when code like this repeats in 2 or more places throughout a program, it is a good idea to isolate it to a method and call that method instead. For example, we could write a function

```python
def _advance_wrap(self, row:int, col:int) -> (int, int):
    return (col+1) % self._n,
           (row+1) % self._n if front_col == 0 else row
```

This will simplify the rest of the program significantly. Methods like:

```python
    def __iter__(self):
        """Yield occupants from front to back in FIFO order (handles wrap-around)."""
        count = 0
        row, col = self._front
        while count < self._usage:
            value = self._underlying[row][col]
            # Occupied cells are non-None by invariant; guard defensively anyway.
            if value is not None:
                yield value
            col = (col + 1) % self._n
            row = (row + 1) % self._n if col == 0 else row
            count += 1
```

can be rewritten as:

```python
    def __iter__(self):
        """Yield occupants from front to back in FIFO order (handles wrap-around)."""
        count = 0
        row, col = self._front
        while count < self._usage:
            value = self._underlying[row][col]
            # Occupied cells are non-None by invariant; guard defensively anyway.
            if value is not None:
                yield value
            col, col = self._advance_wrap(row, col) # two lines down to one with simpler appearance
            count += 1
```
